In [ ]:
#import modules
import geopandas as gpd

In [ ]:
#add MTA data to geopandas dataframe
mta_csv = '/Users/davidhugens/Documents/Work/GIS/Data/NYC/MTA_Subway_Stations_20240117.csv'
df_mta = gpd.read_file(mta_csv)
geometry = gpd.points_from_xy(df_mta['GTFS Longitude'], df_mta['GTFS Latitude'])
gdf_mta = gpd.GeoDataFrame(df_mta, geometry=geometry, crs=4326)
gdf_mta = gdf_mta.to_crs(epsg=2263)
gdf_mta.head(3)

In [ ]:
#add Hospitals data to geopandas dataframe
hospitals_csv = '/Users/davidhugens/Documents/Work/GIS/Data/NYC/Hospitals.csv'
df_hosp = gpd.read_file(hospitals_csv)
geometry = gpd.points_from_xy(df_hosp['long'], df_hosp['lat'])
gdf_hosp = gpd.GeoDataFrame(df_hosp, geometry=geometry, crs=4326)
gdf_hosp = gdf_hosp.to_crs(epsg=2263)
gdf_hosp.head(3)

In [ ]:
#limit MTA gdf to only include ada accessible stations
mta_ada = gdf_mta[gdf_mta['ADA'] == '1']
#visual check
mta_ada.head(3)

In [ ]:
#join nearest accessible mta station to hospitals and look at the result
hosp_join = gpd.sjoin_nearest(gdf_hosp,mta_ada, how='inner',distance_col="dist")
hosp_join['dist_int'] = hosp_join['dist'].astype(int)
hosp_acc = hosp_join[hosp_join['dist_int'] <= 1320]
hosp_acc = hosp_acc.to_crs(epsg=4326)


In [ ]:
#look at the map
nyc = gpd.read_file("/Users/davidhugens/Documents/Work/GIS/Data/NYC/BoroughBoundaries.geojson")
if nyc.crs != gdf_hosp.crs:
    gdf_hosp = gdf_hosp.to_crs(nyc.crs)
if nyc.crs != hosp_acc.crs:
    hosp_acc = hosp_acc.to_crs(nyc.crs)
ax = nyc.plot(alpha = 0.5, legend=True)
gdf_hosp.plot(ax=ax, facecolor = 'r', label = "All Hospitals")
hosp_acc.plot(ax=ax, facecolor = 'g', label = "Hospitals within 1/4 mile of ADA Accessible Subway")
ax.set_title('NYC Public Hospital Accessibility')
ax.set_axis_off()
ax.legend(bbox_to_anchor=(0, -.001), loc='upper left')